# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [7]:
# your code goes here

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")

filtered_df = df[(df["Total Claim Amount"] < 1000) & (df["Response"] == "Yes")]

grouped_avg = filtered_df.groupby(["Policy Type", "Gender"])[
    ["Monthly Premium Auto", "Customer Lifetime Value"]
].mean().reset_index()

grouped_claim = filtered_df.groupby(["Policy Type", "Gender"])[
    "Total Claim Amount"
].mean().reset_index().rename(columns={"Total Claim Amount": "Avg Total Claim Amount"})

profitability_df = pd.merge(grouped_avg, grouped_claim, on=["Policy Type", "Gender"])

print(profitability_df)

      Policy Type Gender  Monthly Premium Auto  Customer Lifetime Value  \
0  Corporate Auto      F             89.042945              7334.770760   
1  Corporate Auto      M             88.546667              7920.399363   
2   Personal Auto      F             90.620079              7966.926671   
3   Personal Auto      M             87.433852              7481.817882   
4    Special Auto      F             86.705882              7594.009218   
5    Special Auto      M             80.000000              8348.232787   

   Avg Total Claim Amount  
0              407.800039  
1              388.247325  
2              404.972992  
3              427.868684  
4              426.658992  
5              390.329804  


In [8]:
state_counts = df["State"].value_counts()
states_over_500 = state_counts[state_counts > 500]
print(states_over_500)

State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64


In [9]:
clv_stats = df.groupby(["Education", "Gender"])["Customer Lifetime Value"].agg(["max", "min", "median"]).reset_index()
print(clv_stats)

              Education Gender          max          min       median
0              Bachelor      F  73225.95652  1904.000852  5640.505303
1              Bachelor      M  67907.27050  1898.007675  5548.031892
2               College      F  61850.18803  1898.683686  5623.611187
3               College      M  61134.68307  1918.119700  6005.847375
4                Doctor      F  44856.11397  2395.570000  5332.462694
5                Doctor      M  32677.34284  2267.604038  5577.669457
6  High School or Below      F  55277.44589  2144.921535  6039.553187
7  High School or Below      M  83325.38119  1940.981221  6286.731006
8                Master      F  51016.06704  2417.777032  5729.855012
9                Master      M  50568.25912  2272.307310  5579.099207
